In [1]:
import pandas as pd
import os
import sys
import re

In [2]:
file_name2= r"/Users/suranjanachowdhury/Downloads/Final_2023_01_07_Mutual_Fund_Data_based_on_SEC.xlsx"
#r"/Users/suranjanachowdhury/Downloads/Innovation Keywords_Dupl_Str_risk.xlsx"
data_raw=pd.read_excel(file_name2)
print(data_raw.shape)

(183111, 26)


In [3]:
import nltk
#nltk.download()

In [4]:
sent_col_name = 'risk'
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: str(x).replace("&nbsp;", " "))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\\", ''))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\t", ''))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\n", ''))

In [5]:
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [6]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [7]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    
    #getting all the keys for abbrevations
    contractions = abbreviations.keys()
    
    #making the list of sentence terminators
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    
    for sentence_terminator in sentence_terminators:
    
        #Finding the position of sentence terminators
        t_indices = list(find_all(paragraph, sentence_terminator))
        
        #Finding the length of sentence terminators
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    
    for contraction in contractions:
        
        #Finding the position of abbrevations defined above
        c_indices = list(find_all(paragraph, contraction))
        
        #Finding the end of abbrevations in a disclosure                  
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    
    #tokenizing the sentences from paragraph
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    
    return end

In [8]:
def find_sentences(paragraph):
    
    replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], 
                     ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], 
                     ["S. E. C.", "S.E.C."]]
    
    for items in replacingList:
        #adding space or cleaning the common abbrevations
        paragraph = paragraph.replace(items[0], items[1])
    
    #replacing all the extra spaces with single space    
    paragraph = re.sub(' +', ' ', paragraph)
    
    #removing the abbrevations which are used to denote staement ending from the beginning of the statement
    if paragraph != "":
        if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
            paragraph = paragraph[1:]    
    
    end = True
    sentences = []
    
    while end > -1:
        end = find_sentence_end(paragraph)
        if end > -1:
            sentences.append(paragraph[end:].strip())
            paragraph = paragraph[:end]
    
    sentences.append(paragraph)
    sentences.reverse()
    
    return sentences

In [9]:
#import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data_raw['cleaned_principal_risks']= ""

for row in data_raw.itertuples():
    if row.Index %5000 == 0:
        print ("processing row: ", row.Index)    
        
    para = row.risk
    para = para.replace('{',' ')
    para = para.replace('}',' ')
    sents = find_sentences(para)
    text = ' '.join(sents)
    text = re.sub('[^,.a-zA-Z0-9 ]',' ', text)
    text = '\n'.join(tokenizer.tokenize(text))
    data_raw['cleaned_principal_risks'].loc[row.Index] = text

processing row:  0


/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_15217/2922097148.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_raw['cleaned_principal_risks'].loc[row.Index] = text


processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000
processing row:  110000
processing row:  115000
processing row:  120000
processing row:  125000
processing row:  130000
processing row:  135000
processing row:  140000
processing row:  145000
processing row:  150000
processing row:  155000
processing row:  160000
processing row:  165000
processing row:  170000
processing row:  175000
processing row:  180000


In [10]:
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
import os
import pandas as pd
import glob
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters, PunktTrainer

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/suranjanachowdhury/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
# Tokenizing disclosure text
data_raw['tokens'] = data_raw.apply(lambda row: nltk.sent_tokenize(row['cleaned_principal_risks']), axis=1)

In [13]:
data_raw = data_raw.reset_index()

In [14]:
data_raw.head()

,index,Unnamed: 0,adsh,series,cik,ddate,pdate,effdate,Fund_name_from_SEC_filing,country_business_address,...,zip_mailing_address,mailing_address_1,mailing_address_2,strategy,risk,crsp_class,Fund_name_from_CRSP,year,cleaned_principal_risks,tokens
0,0,0,0001193125-10-283645,S000004351,1100663,20101130,20101201.0,20101201.0,ISHARES TRUST,US,...,2116,200 CLARENDON,NaN,The Underlying Index has been developed by MSC...,"As with any investment, you could lose all or ...",FE,iShares Trust: iShares MSCI EAFE ETF,2010,As with any investment you could lose all or p...,[As with any investment you could lose all or ...
1,1,1,0001193125-11-342625,S000004351,1100663,20111130,20111201.0,20111201.0,ISHARES TRUST,US,...,02116,200 CLARENDON,NaN,The Underlying Index has been developed by MSC...,"As with any investment, you could lose all or ...",FE,iShares Trust: iShares MSCI EAFE ETF,2011,As with any investment you could lose all or p...,[As with any investment you could lose all or ...
2,2,2,0001193125-12-505158,S000004351,1100663,20121130,20121201.0,20121201.0,ISHARES TRUST,US,...,94105,400 HOWARD STREET,NaN,The Underlying Index has been developed by MSC...,"As with any investment, you could lose all or ...",FE,iShares Trust: iShares MSCI EAFE ETF,2012,As with any investment you could lose all or p...,[As with any investment you could lose all or ...
3,3,3,0001193125-13-252030,S000004351,1100663,20130531,20130523.0,20130524.0,ISHARES TRUST,US,...,94105,400 HOWARD STREET,NaN,The Underlying Index has been developed by MSC...,"As with any investment, you could lose all or ...",FE,iShares Trust: iShares MSCI EAFE ETF,2013,As with any investment you could lose all or p...,[As with any investment you could lose all or ...
4,4,4,0001193125-13-297569,S000004351,1100663,20130630,20130701.0,20130701.0,ISHARES TRUST,US,...,94105,400 HOWARD STREET,NaN,The Underlying Index has been developed by MSC...,"As with any investment, you could lose all or ...",FE,iShares Trust: iShares MSCI EAFE ETF,2013,As with any investment you could lose all or p...,[As with any investment you could lose all or ...


In [15]:
inflation_words = ['innovation','innovative','innovate','disruptive','technologies','technology','technical','tech']

In [16]:
keyword_sentence = []


for row in data_raw.itertuples():
    if row.Index %5000 == 0:
        print ("processing row: ", row.Index)        
   
    for j in row.tokens:
        for k in inflation_words:
            if k in j:
#               print(row.accession,row.filing_date, row.crsp_class, row.filing_year, j)
                #keyword_sentence.append([row.accession,row.filing_date, row.crsp_class, row.filing_year, j])
                keyword_sentence.append([row.series, 
                                         row.ddate,
                                         row.year,
                                         row.crsp_class,
                                         #row.crsp_fundno, 
                                         #row.crsp_portno, 
                                         #row.fund_name, 
                                         row.Fund_name_from_SEC_filing,
                                         #row.comp_cik,                                      
                                         row.cik,
                                         j,
                                         k]) 
 

processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000
processing row:  110000
processing row:  115000
processing row:  120000
processing row:  125000
processing row:  130000
processing row:  135000
processing row:  140000
processing row:  145000
processing row:  150000
processing row:  155000
processing row:  160000
processing row:  165000
processing row:  170000
processing row:  175000
processing row:  180000


In [17]:
print(len(keyword_sentence))

229027


In [18]:
df_sent = pd.DataFrame(keyword_sentence, columns =['series',                                               
                                                   'ddate',
                                                   'year',
                                                   'crsp_class',
                                                   #'crsp_fundno', 
                                                   #'crsp_portno', 
                                                   #'fund_name', 
                                                   'Fund_name_from_SEC_filing',
                                                   'comp_cik',
                                                   'sentences',
                                                   'keyword_contained'])


In [19]:
df_sent['crsp_class'].value_counts()

others     64028
DE         58647
Missing    44832
FE         37870
M          13777
FI          8564
Index       1309
Name: crsp_class, dtype: int64

In [20]:
#df_sent1 = df_sent.drop_duplicates('sentences')

In [21]:
#df_sent1.shape

In [22]:
df_sent.shape

(229027, 8)

In [23]:
df_sent.head()

,series,ddate,year,crsp_class,Fund_name_from_SEC_filing,comp_cik,sentences,keyword_contained
0,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology
1,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,tech
2,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risk arisin...,technology
3,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risk arisin...,tech
4,S000004351,20160731,2016,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology


In [24]:
df_sent_unique = df_sent.drop_duplicates(subset =['sentences'])
print(df_sent_unique.shape)
df_sent_unique.head()

(8030, 8)


,series,ddate,year,crsp_class,Fund_name_from_SEC_filing,comp_cik,sentences,keyword_contained
0,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology
2,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risk arisin...,technology
8,S000004351,20161130,2016,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology
10,S000004351,20161130,2016,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risks arisi...,technology
12,S000004351,20171130,2017,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology


In [25]:
df_sent.head()

,series,ddate,year,crsp_class,Fund_name_from_SEC_filing,comp_cik,sentences,keyword_contained
0,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology
1,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,tech
2,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risk arisin...,technology
3,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risk arisin...,tech
4,S000004351,20160731,2016,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology


In [26]:
df_sent.shape

(229027, 8)

In [27]:
df_sent['crsp_class'].value_counts()

others     64028
DE         58647
Missing    44832
FE         37870
M          13777
FI          8564
Index       1309
Name: crsp_class, dtype: int64

# Unit of sentences to unit of series

In [28]:
# disclosure label

In [29]:
#commenting below as getting error
#df = df_inflation_with_label_1.copy()
all_series = df_sent['series'].unique()
df=df_sent.copy()

In [30]:
print(len(all_series))

12373


In [31]:
all_series

array(['S000004351', 'S000004352', 'S000004353', ..., 'S000079342',
       'S000079345', 'S000079362'], dtype=object)

In [32]:
df.head()

,series,ddate,year,crsp_class,Fund_name_from_SEC_filing,comp_cik,sentences,keyword_contained
0,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology
1,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,tech
2,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risk arisin...,technology
3,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,The Fund is exposed to operational risk arisin...,tech
4,S000004351,20160731,2016,FE,ISHARES TRUST,1100663,In recent years cyber attacks and technology m...,technology


In [33]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
series_info = []
for series in all_series:  # for each series
    df_temp = df[df['series'] == series]

    all_ddate = df_temp['ddate'].unique()

    for ddate in all_ddate:  # for a given series, iterate all filing dates
        df_temp_ddate = df_temp[df_temp['ddate'] == ddate]

        label_temp = 'innovation'

        all_sentences = []
        relevant_to_CC = 0

        if len(df_temp_ddate) == 0:
            print (' error ....')
            break

        for row_temp in range(len(df_temp_ddate)):
            all_sentences.append(df_temp_ddate['sentences'
                                 ].iloc[row_temp])

            # df_temp_ddate.head()

        for row_temp in range(len(df_temp_ddate)):

            label1 = df_temp_ddate['keyword_contained'].iloc[row_temp]
            if label1 == 'innovative':
                label_temp = 'innovative'
                print ('here 1')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                # crsp_fundno_temp=  df_temp_ddate['crsp_fundno'].iloc[row_temp]
                # crsp_portno_temp=  df_temp_ddate['crsp_portno'].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]

                # series

                break
            elif label1 == 'innovation':
                label_temp = 'innovation'
                print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                # crsp_fundno_temp=  df_temp_ddate['crsp_fundno'].iloc[row_temp]
                # crsp_portno_temp=  df_temp_ddate['crsp_portno'].iloc[row_temp]
                # fund_name_temp= df_temp_ddate['fund_name'].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break
            elif label1 == 'development':
                label_temp = 'development'
                print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                # crsp_fundno_temp=  df_temp_ddate['crsp_fundno'].iloc[row_temp]
                # crsp_portno_temp=  df_temp_ddate['crsp_portno'].iloc[row_temp]
                # fund_name_temp= df_temp_ddate['fund_name'].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break  
            elif label1 == 'disruptive':
                label_temp = 'disruptive'
                print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                # crsp_fundno_temp=  df_temp_ddate['crsp_fundno'].iloc[row_temp]
                # crsp_portno_temp=  df_temp_ddate['crsp_portno'].iloc[row_temp]
                # fund_name_temp= df_temp_ddate['fund_name'].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break  
            elif label1 == 'technologies':
                label_temp = 'technologies'
                print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                # crsp_fundno_temp=  df_temp_ddate['crsp_fundno'].iloc[row_temp]
                # crsp_portno_temp=  df_temp_ddate['crsp_portno'].iloc[row_temp]
                # fund_name_temp= df_temp_ddate['fund_name'].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break 
               # 'technologies','technology','technical','tech'
            elif label1 == 'tech':
                label_temp = 'tech'
                #print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break 
            elif label1 == 'technology':
                label_temp = 'technology'
                #print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break 
            elif label1 == 'technical':
                label_temp = 'technical'
                #print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break     
            elif label1 == 'investment':
                label_temp = 'investment'
                print ('here 2')
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                # crsp_fundno_temp=  df_temp_ddate['crsp_fundno'].iloc[row_temp]
                # crsp_portno_temp=  df_temp_ddate['crsp_portno'].iloc[row_temp]
                # fund_name_temp= df_temp_ddate['fund_name'].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break     
            else:

                # (label1 == 'innovate') :

                #print ('here 3')
                label_temp = 'innovate'
                series = df_temp_ddate['series'].iloc[row_temp]
                date_temp = df_temp_ddate['ddate'].iloc[row_temp]
                year_temp = df_temp_ddate['year'].iloc[row_temp]
                crsp_class_temp = df_temp_ddate['crsp_class'
                        ].iloc[row_temp]

                # crsp_fundno_temp=  df_temp_ddate['crsp_fundno'].iloc[row_temp]
                # crsp_portno_temp=  df_temp_ddate['crsp_portno'].iloc[row_temp]
                # fund_name_temp= df_temp_ddate['fund_name'].iloc[row_temp]

                fund_name_temp = \
                    df_temp_ddate['Fund_name_from_SEC_filing'
                                  ].iloc[row_temp]
                comp_cik_temp = df_temp_ddate['comp_cik'].iloc[row_temp]
                break
        series_info.append([  # crsp_fundno_temp,
                              # crsp_portno_temp,
            series,
            date_temp,
            year_temp,
            crsp_class_temp,
            fund_name_temp,
            comp_cik_temp,
            all_sentences,
            label_temp,
            ])


here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2

here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2

here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1

here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2

here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2

here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1

here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 1
here 1
here 1
here 1
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2
here 2

In [34]:
print(series_info)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [35]:
df_series_labels = pd.DataFrame(series_info, columns =['series', 
                                                   'ddate',
                                                   'year',
                                                   'crsp_class',
                                                   #'crsp_fundno', 
                                                   #'crsp_portno', 
                                                   'Fund_name_from_SEC_filing',
                                                   'comp_cik', 
                                                   'combined_sentences',
                                                   'fund_series_label'])

In [36]:
df_series_labels.head()

,series,ddate,year,crsp_class,Fund_name_from_SEC_filing,comp_cik,combined_sentences,fund_series_label
0,S000004351,20151130,2015,FE,ISHARES TRUST,1100663,[In recent years cyber attacks and technology ...,technology
1,S000004351,20160731,2016,FE,ISHARES TRUST,1100663,[In recent years cyber attacks and technology ...,technology
2,S000004351,20161130,2016,FE,ISHARES TRUST,1100663,[In recent years cyber attacks and technology ...,technology
3,S000004351,20171130,2017,FE,ISHARES TRUST,1100663,[In recent years cyber attacks and technology ...,technology
4,S000004351,20181130,2018,FE,ISHARES TRUST,1100663,[In recent years cyber attacks and technology ...,technology


In [37]:
df_series_labels.shape

(73418, 8)

In [38]:
df_temp_ddate.head()

,series,ddate,year,crsp_class,Fund_name_from_SEC_filing,comp_cik,sentences,keyword_contained
229021,S000079362,20221231,2022,Missing,COLLABORATIVE INVESTMENT SERIES TRUST,1719812,The technology sector can be significantly aff...,technology
229022,S000079362,20221231,2022,Missing,COLLABORATIVE INVESTMENT SERIES TRUST,1719812,The technology sector can be significantly aff...,tech
229023,S000079362,20221231,2022,Missing,COLLABORATIVE INVESTMENT SERIES TRUST,1719812,In addition information technology industries ...,technology
229024,S000079362,20221231,2022,Missing,COLLABORATIVE INVESTMENT SERIES TRUST,1719812,In addition information technology industries ...,tech
229025,S000079362,20221231,2022,Missing,COLLABORATIVE INVESTMENT SERIES TRUST,1719812,The communication services sector can be signi...,technology


In [39]:
df_series_labels['fund_series_label'].value_counts()

tech            42012
technology      20343
technologies     3834
technical        3113
disruptive       2009
innovative       1105
innovation       1000
innovate            2
Name: fund_series_label, dtype: int64

In [40]:
df_series_labels['innovation'] = 0
df_series_labels['innovative'] = 0
df_series_labels['innovate'] = 0
df_series_labels['development']=0
df_series_labels['disruptive']=0
df_series_labels['technologies']=0
df_series_labels['technology']=0
df_series_labels['tech']=0
df_series_labels['technical']=0
df_series_labels['investment']=0

for row in range(len(df_series_labels)):

    label1 = df_series_labels['fund_series_label'].iloc[row]
    if label1 =='innovation':   
        #print(label1)
        df_series_labels['innovation'].iloc[row] = 1 
    if label1 =='innovative':
        df_series_labels['innovative'].loc[row] = 1 
    if label1 =='innovate':
        df_series_labels['innovate'].loc[row] = 1 
    if label1 =='development':
        df_series_labels['development'].loc[row] = 1 
    if label1 =='disruptive':
        df_series_labels['disruptive'].loc[row] = 1 
    if label1 =='investment':
        df_series_labels['investment'].loc[row] = 1 
    if label1 =='technologies':
        df_series_labels['technologies'].loc[row] = 1 
    if label1 =='technology':
        df_series_labels['technology'].loc[row] = 1 
    if label1 =='tech':
        df_series_labels['tech'].loc[row] = 1 
    if label1 =='techical':
        df_series_labels['technical'].loc[row] = 1     
df_series_labels['innovation'].value_counts()  

/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_15217/3795469800.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_series_labels['technology'].loc[row] = 1
/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_15217/3795469800.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_series_labels['tech'].loc[row] = 1
/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_15217/3795469800.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

0    72418
1     1000
Name: innovation, dtype: int64

In [41]:
print(df_series_labels['fund_series_label'].value_counts())
print(df_series_labels['innovation'].value_counts())
print(df_series_labels['innovative'].value_counts())
print(df_series_labels['innovate'].value_counts())
print(df_series_labels['technologies'].value_counts())
print(df_series_labels['tech'].value_counts())
print(df_series_labels['technical'].value_counts())
print(df_series_labels['development'].value_counts())
print(df_series_labels['disruptive'].value_counts())
print(df_series_labels['investment'].value_counts())


tech            42012
technology      20343
technologies     3834
technical        3113
disruptive       2009
innovative       1105
innovation       1000
innovate            2
Name: fund_series_label, dtype: int64
0    72418
1     1000
Name: innovation, dtype: int64
0    72313
1     1105
Name: innovative, dtype: int64
0    73416
1        2
Name: innovate, dtype: int64
0    69584
1     3834
Name: technologies, dtype: int64
1    42012
0    31406
Name: tech, dtype: int64
0    73418
Name: technical, dtype: int64
0    73418
Name: development, dtype: int64
0    71409
1     2009
Name: disruptive, dtype: int64
0    73418
Name: investment, dtype: int64


In [42]:
df2 = df_series_labels.copy()
with pd.ExcelWriter('NewKeywordTest2.xlsx') as writer:  
   # df1.to_excel(writer, sheet_name='Sheet_name_1')
    df2.to_excel(writer, sheet_name='Risk')